In [ ]:
from pymongo import MongoClient

client = MongoClient()
test_collection = client["test"]["comment"]

In [71]:
import requests
from urllib import parse
import re
import json


def getJson(url, query={}, init_api_url="https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&pool=cbox3&lang=ko&objectId=183559_517&pageSize=100&page=1&sort=NEW"):    
    # make valid header
    parsed_url = parse.urlparse(url)
    parsed_url_query = dict(parse.parse_qsl(parsed_url.query))
    for key in list(parsed_url_query.keys()): # remove weekday value
        if key not in ("titleId", "no"):
            parsed_url_query.pop(key)
    comment_url = parse.urlunparse(parsed_url._replace(path="/comment/comment", query=parse.urlencode(parsed_url_query)))
    header = {"referer":comment_url,
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36"}
    print(header)

    # get api url using query of init api url
    parsed_init_api_url = parse.urlparse(init_api_url)
    parsed_init_api_url_query = dict(parse.parse_qsl(parsed_init_api_url.query))
    new_objectId = "_".join(parsed_url_query.values())
    print(new_objectId)
    parsed_init_api_url_query["objectId"] = new_objectId # change only objectId query value
    for key, value in query.items():
        parsed_init_api_url_query[key] = value
    api_url = parse.urlunparse(parsed_init_api_url._replace(query=parse.urlencode(parsed_init_api_url_query)))
    print(api_url)
    
    # removing useless prefix and subfix
    removeElse = re.compile("^[^{]+|[^}]+$")
    api_json = json.loads(removeElse.sub("", requests.get(api_url, headers=header).text))
    return api_json

In [64]:
url = "https://comic.naver.com/webtoon/detail?titleId=783054&no=62&weekday=tue"
init_test_api_url="https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic_challenge&templateId=default&pool=cbox3&_wr&_callback=jQuery1124034503790000299817_1644289480843&lang=ko&country=KR&objectId=701701_1176&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&pageType=default&page=1&refresh=true&sort=NEW&_=1644289480845"
url_json=getJson(url)

{'referer': 'https://comic.naver.com/comment/comment?titleId=783054&no=62', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}
783054_62
https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&pool=cbox3&lang=ko&objectId=783054_62&pageSize=100&page=1&sort=sim


In [ ]:
from datetime import datetime


dateFormat = "%Y-%m-%dT%H:%M:%S%z"
while True:
    target_url = "https://comic.naver.com/webtoon/detail?titleId=783054&no=62&weekday=tue"
    pagenum = getJson(target_url)["result"]["pageModel"]["lastPage"]
    comment_data = []
    while pagenum > 0:
        comment_json = getJson(target_url, {"page":pagenum})
        comment_list = url_json["result"]["commentList"][::-1]
        for comment in comment_list:
            if datetime.strptime(comment["regTime"]):

                pass
                # print(i["userIdNo"], i["regTime"], i["sympathyCount"], i["antipathyCount"])
        break
    break
            

In [75]:
from datetime import datetime


datetime.strptime("2022-02-07T22:53:29+0900", "%Y-%m-%dT%H:%M:%S%z")

datetime.datetime(2022, 2, 7, 22, 53, 29, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400)))

In [65]:
from pprint import pprint


pprint(url_json)
comment_list = None
if url_json["success"]:
    comment_list = url_json["result"]["commentList"]
len(comment_list)

{'code': '1000',
 'country': 'UNKNOWN',
 'date': '2022-02-08T04:21:09+0000',
 'lang': 'ko',
 'message': '요청을 성공적으로 처리하였습니다.',
 'result': {'bestList': [],
            'commentList': [{'anonymous': False,
                             'antipathy': False,
                             'antipathyCount': 20,
                             'best': True,
                             'blind': False,
                             'blindReport': False,
                             'categoryId': '*',
                             'categoryImage': None,
                             'commentNo': '430650831',
                             'commentType': 'txt',
                             'containText': True,
                             'contents': '갑수할배 어디감?? \n자연사했나??',
                             'country': 'KR',
                             'defamation': False,
                             'deleted': False,
                             'expose': True,
                             'exposeByCountry': F

13

In [55]:
total_text = ""
text_cleaning = re.compile("[^가-힣 ]")

for i in comment_list:
    print("---------------------------------------")
    print(i["userIdNo"], i["regTime"], i["sympathyCount"], i["antipathyCount"])
    print(i["contents"])
    total_text += " " + text_cleaning.sub(" ", i["contents"])
    if i["modTime"] != i["regTime"]:
        print(i)
        break

---------------------------------------
2aF5U 2022-02-08T12:23:03+0900 0 0
ㅋㅋㅋㅋㅋ
---------------------------------------
d7xLp 2022-02-08T12:08:12+0900 0 0
힙합은 어디로 갔냐?
---------------------------------------
bIEwX 2022-02-08T11:57:09+0900 0 0
아 존ㄴr 웃기네 진짜 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
---------------------------------------
1Twk8 2022-02-08T11:54:42+0900 0 0
이번화 개웃기네ㅋㅋㅋ
---------------------------------------
4T8kn 2022-02-08T11:41:18+0900 0 0
중국사라져라
---------------------------------------
VjfC 2022-02-08T11:39:42+0900 0 0
zzzzzzzzzzzzzzzzzz
---------------------------------------
3gLYk 2022-02-08T11:15:23+0900 0 0
으 진짜 재믹자 ㅋ   ㅋ ㅋ 아 가웃겨,,!!!
---------------------------------------
3j7XH 2022-02-08T11:12:22+0900 0 0
연막탄 던지자마자 귀신같이 성추행하노 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
---------------------------------------
cTJ2 2022-02-08T11:09:07+0900 0 0
강건마 VS 설전설 누가 더 OOO일까?
---------------------------------------
cWkCF 2022-02-08T10:59:02+0900 0 0
ㄷㄷㄷㄷㄷ
---------------------------------------
cGeSy 2022-02-

In [ ]:
total_text

In [ ]:
from konlpy.tag import Okt
from collections import Counter

noun_counter = Counter(Okt().nouns(total_text))

In [ ]:
noun_counter

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud = WordCloud(font_path='NanumGothic', background_color='white', colormap="cool").fit_words(noun_counter)
plt.imshow(wordcloud)
plt.axis("off")
# wordcloud.to_image().show()
# wordcloud.to_file('test.jpg')

In [ ]:
total_text